In [1]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import jsonlines
import pandas
import jieba
import re
import embedding
import numpy

In [2]:
embedding.mergeDict() #生成字典，不需要执行

## 下面都是处理数据

In [ ]:
data_web = pandas.read_excel(r'D:\repositories\DaChuang\data\news_each_school\info_output.xlsx')

In [ ]:
data_wx = pandas.read_csv(r'D:\repositories\DaChuang\data\公众号\ruc_info.csv')

In [ ]:
#丢弃nan
data_wx.dropna(axis=0,inplace=True,how='any')
data_web.dropna(axis=0,inplace=True,how='any')

In [ ]:
#重新排index
data_wx.reset_index(drop=True, inplace=True)
data_web.reset_index(drop=True, inplace=True)

In [ ]:
#将dataframe中的换行全都删去（公众号内容中有很多）
for index,content in enumerate(data_wx['content']):
    data_wx.loc[index,'content'] = re.sub('\n|\r','',content)

for index,content in enumerate(data_web['正文']):
    data_web.loc[index,'正文'] = re.sub('\n|\r','',content)

for index,content in enumerate(data_wx['title']):
    data_wx.loc[index,'title'] = re.sub('\n|\r','',content)

for index,content in enumerate(data_web['标题']):
    data_web.loc[index,'标题'] = re.sub('\n|\r','',content)

In [ ]:
#拼接
data = pandas.concat([data_web.正文,data_wx.content,data_web.标题,data_wx.title])

## 以下是训练word2vec的内容

In [ ]:
stop = embedding.getStopList()

In [ ]:
corpus = embedding.getCorpus() 获得语料

In [ ]:
jieba.load_userdict('../data/语料/dictionary.txt')

In [ ]:
corpus=[]
for sentence in data:
    #line = []
    segment = jieba.cut(sentence)
    seg = [word for word in list(segment) if word not in stop]
    
    #for word in simple_preprocess(seg,deacc=True,min_len=1):
    #    if word not in stop:
    #        line.append(word)
    corpus.append(seg)     

In [ ]:
with open(r'D:\repositories\DaChuang\data\teachers\teachers_information.json','r',encoding='utf-8') as f:
    for teacher in jsonlines.Reader(f):
        line = []
        line.append(teacher['department'])
        line.append(teacher['name'])
        try:
            line+=teacher['major']
        except:
            pass
        corpus.append(line)

In [ ]:
#输出语料
g = open(r'D:\codes\Pt_MarkDown\大创\data\corpus_information.txt','w',encoding='utf-8')
for sentence in corpus:
    line = ' '.join(sentence)
    g.write(line+'\n')
g.close()

In [ ]:
#统计关键词数量小于2的文档数
dictionary = embedding.getDict()
count_g = 0
docs = []
for doc in corpus:
    count = 0
    for word in doc:
        if word in dictionary:
            count+=1
    if count < 2:
        docs.append(doc)
        count_g += 1

In [ ]:
docs[2]

In [41]:
#训练模型
w2v_model = Word2Vec(corpus, size=100, min_count=1, workers=16)

In [42]:
w2v_model.save(r'D:\codes\Pt_MarkDown\大创\data\word_embeddings_information.model')

In [43]:
w2v_model.wv.similarity('窦志成','信息检索')

0.5212252

In [44]:
w2v_model.most_similar('信息学院')

[('我院', 0.6962107419967651),
 ('全院', 0.6765086650848389),
 ('秋冬季', 0.6276863813400269),
 ('座谈会', 0.6230578422546387),
 ('第六次', 0.6109772324562073),
 ('一个系列', 0.608765721321106),
 ('表示祝贺', 0.6076531410217285),
 ('数学系', 0.6065365076065063),
 ('人民大学', 0.5948217511177063),
 ('顺利', 0.5932497978210449)]

In [52]:
w2v_model.most_similar('微软')

[('亚洲', 0.920695960521698),
 ('研究院', 0.8535282611846924),
 ('工程院', 0.8212538957595825),
 ('JasonWu', 0.8192484974861145),
 ('InfoSys', 0.815918505191803),
 ('IBM', 0.7860386371612549),
 ('MSRA', 0.7722996473312378),
 ('QUIQ', 0.769305944442749),
 ('Pivotal', 0.7630451917648315),
 ('经理人', 0.7597249746322632)]

In [46]:
w2v_model.most_similar('深度学习')

[('知识图谱', 0.9517089128494263),
 ('隐私保护', 0.9451524019241333),
 ('自然语言处理', 0.9438310861587524),
 ('机器学习', 0.9416126012802124),
 ('信息网络', 0.9404984712600708),
 ('隐私', 0.93840491771698),
 ('高频', 0.9383750557899475),
 ('推荐系统', 0.9352732300758362),
 ('社会网络', 0.9343469738960266),
 ('挑战性', 0.9322288036346436)]

In [47]:
w2v_model.most_similar('宣讲')

[('交流平台', 0.756393551826477),
 ('调研', 0.7418028712272644),
 ('面对面', 0.7341629266738892),
 ('全程参加', 0.7308314442634583),
 ('开展调研', 0.7291984558105469),
 ('日常化', 0.7213379144668579),
 ('近期', 0.7122913002967834),
 ('互动', 0.708523690700531),
 ('形式', 0.7075490951538086),
 ('征集', 0.7058008909225464)]

In [48]:
w2v_model.most_similar('明德楼')

[('层', 0.9309592247009277),
 ('一层', 0.9287614822387695),
 ('1001', 0.9282403588294983),
 ('游泳馆', 0.9264535903930664),
 ('6308', 0.9256508946418762),
 ('大厦', 0.9253577589988708),
 ('北', 0.9230436086654663),
 ('主楼', 0.9199074506759644),
 ('二楼', 0.9188681840896606),
 ('汇贤', 0.9127611517906189)]

In [49]:
w2v_model.most_similar('SIGIR')

[('WWW', 0.9669920802116394),
 ('MSST', 0.9549739956855774),
 ('TKDE', 0.9537514448165894),
 ('CIKM', 0.9516314268112183),
 ('ICDE', 0.9500011205673218),
 ('EDBT', 0.9492573738098145),
 ('EI', 0.9484114646911621),
 ('AAAI', 0.9479264616966248),
 ('P', 0.9476830959320068),
 ('WSDM', 0.94678795337677)]